

# Random Forest

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import SMOTE

In [3]:
df = pd.read_csv('/content/sample_data/final.csv')

In [ ]:
features = ['gender', 'RACE_Asian', 'RACE_Black/African American',
            'RACE_Hispanic/Latino', 'RACE_Other', 'RACE_White', 'age_18-24',
            'age_25-44', 'age_45-64', 'age_65-88',
            'age_89+', 'previous_stays', 'prev_los_avg', 'Anion Gap',
            'Bicarbonate', 'Chloride', 'Creatinine', 'Glucose', 'Hematocrit',
            'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Platelet Count',
            'Potassium', 'RDW', 'Red Blood Cells', 'Sodium', 'Urea Nitrogen',
            'White Blood Cells', 'prescrip_count', 'diagnoses_num',
            'ADM_Emergency', 'ADM_Other', 'ADM_Referral', 'ADM_Transfer',
            'INS_Medicaid', 'INS_Medicare', 'INS_Other']


X = df[features]
y = df['los_category']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
rf_model = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 6, 9],
    'max_features': [0.6, 0.7, 0.8]
}
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train_resampled, y_train_resampled)
print(f"Best parameters found: {grid_search.best_params_}")

best_rf_model = grid_search.best_estimator_
best_rf_model.fit(X_train_resampled, y_train_resampled)
y_pred_rf = best_rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rf)
print("Overall Accuracy of Random Forest Classifier:", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred_rf)
print("Overall Confusion Matrix:")
print(conf_matrix)

label_encoder = LabelEncoder()
label_encoder.fit(y_encoded)

y_test_labels = label_encoder.inverse_transform(y_test)
y_pred_labels = label_encoder.inverse_transform(y_pred_rf)

class_report_labels = classification_report(y_test_labels, y_pred_labels)
print("Overall Classification Report:")
print(class_report_labels)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/Users/ashademeij/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/ashademeij/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/ashademeij/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/ashademeij/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' 

[CV] END ....max_depth=3, max_features=0.6, n_estimators=100; total time=   5.2s
[CV] END ....max_depth=3, max_features=0.6, n_estimators=100; total time=   6.4s
[CV] END ....max_depth=3, max_features=0.6, n_estimators=100; total time=   7.2s
[CV] END ....max_depth=3, max_features=0.6, n_estimators=200; total time=  10.3s
[CV] END ....max_depth=3, max_features=0.6, n_estimators=200; total time=  13.1s
[CV] END ....max_depth=3, max_features=0.6, n_estimators=200; total time=  13.8s
[CV] END ....max_depth=3, max_features=0.7, n_estimators=100; total time=   8.1s
[CV] END ....max_depth=3, max_features=0.7, n_estimators=100; total time=   7.7s
[CV] END ....max_depth=3, max_features=0.7, n_estimators=100; total time=   6.1s
[CV] END ....max_depth=3, max_features=0.7, n_estimators=200; total time=  12.4s
[CV] END ....max_depth=3, max_features=0.7, n_estimators=200; total time=  15.9s
[CV] END ....max_depth=3, max_features=0.7, n_estimators=200; total time=  16.6s
[CV] END ....max_depth=3, ma

In [ ]:
def evaluate_subgroup(subgroup_name, subgroup_value):
    subgroup_indices = X_test[subgroup_name] == subgroup_value
    subgroup_X_test = X_test[subgroup_indices]
    subgroup_y_test = y_test[subgroup_indices]
    subgroup_y_pred = y_pred_rf[subgroup_indices]

    accuracy = accuracy_score(subgroup_y_test, subgroup_y_pred)
    print(f"Accuracy for {subgroup_name}={subgroup_value}: {accuracy}")

    conf_matrix = confusion_matrix(subgroup_y_test, subgroup_y_pred)
    print(f"Confusion Matrix for {subgroup_name}={subgroup_value}:")
    print(conf_matrix)

    subgroup_y_test_labels = label_encoder.inverse_transform(subgroup_y_test)
    subgroup_y_pred_labels = label_encoder.inverse_transform(subgroup_y_pred)
    class_report = classification_report(subgroup_y_test_labels, subgroup_y_pred_labels)
    print(f"Classification Report for {subgroup_name}={subgroup_value}:")
    print(class_report)

# gender
print("MALE")
evaluate_subgroup('gender', 0)
print("\nFEMALE")
evaluate_subgroup('gender', 1)

MALE
Accuracy for gender=0: 0.6596942880128721
Confusion Matrix for gender=0:
[[ 607  750]
 [ 519 1853]]
Classification Report for gender=0:
              precision    recall  f1-score   support

           0       0.54      0.45      0.49      1357
           1       0.71      0.78      0.74      2372

    accuracy                           0.66      3729
   macro avg       0.63      0.61      0.62      3729
weighted avg       0.65      0.66      0.65      3729


FEMALE
Accuracy for gender=1: 0.6695212999560826
Confusion Matrix for gender=1:
[[ 701  986]
 [ 519 2348]]
Classification Report for gender=1:
              precision    recall  f1-score   support

           0       0.57      0.42      0.48      1687
           1       0.70      0.82      0.76      2867

    accuracy                           0.67      4554
   macro avg       0.64      0.62      0.62      4554
weighted avg       0.66      0.67      0.66      4554



In [ ]:
# race categories
print("\nASIAN")
evaluate_subgroup('RACE_Asian', 1)
print("\nBLACK/AFRICAN AMERICAN")
evaluate_subgroup('RACE_Black/African American', 1)
print("\nHISPANIC/LATINO")
evaluate_subgroup('RACE_Hispanic/Latino', 1)
print("\nOTHER RACE")
evaluate_subgroup('RACE_Other', 1)
print("\nWHITE")
evaluate_subgroup('RACE_White', 1)



ASIAN
Accuracy for RACE_Asian=1: 0.6982456140350877
Confusion Matrix for RACE_Asian=1:
[[ 42  65]
 [ 21 157]]
Classification Report for RACE_Asian=1:
              precision    recall  f1-score   support

           0       0.67      0.39      0.49       107
           1       0.71      0.88      0.79       178

    accuracy                           0.70       285
   macro avg       0.69      0.64      0.64       285
weighted avg       0.69      0.70      0.68       285


BLACK/AFRICAN AMERICAN
Accuracy for RACE_Black/African American=1: 0.6802030456852792
Confusion Matrix for RACE_Black/African American=1:
[[138 207]
 [108 532]]
Classification Report for RACE_Black/African American=1:
              precision    recall  f1-score   support

           0       0.56      0.40      0.47       345
           1       0.72      0.83      0.77       640

    accuracy                           0.68       985
   macro avg       0.64      0.62      0.62       985
weighted avg       0.66      0.

In [ ]:
#  age groups
print("\nAGE 18-24")
evaluate_subgroup('age_18-24', 1)
print("\nAGE 25-44")
evaluate_subgroup('age_25-44', 1)
print("\nAGE 45-64")
evaluate_subgroup('age_45-64', 1)
print("\nAGE 65-88")
evaluate_subgroup('age_65-88', 1)
print("\nAGE 89+")
evaluate_subgroup('age_89+', 1)


AGE 18-24
Accuracy for age_18-24=1: 0.7553191489361702
Confusion Matrix for age_18-24=1:
[[ 12  37]
 [  9 130]]
Classification Report for age_18-24=1:
              precision    recall  f1-score   support

           0       0.57      0.24      0.34        49
           1       0.78      0.94      0.85       139

    accuracy                           0.76       188
   macro avg       0.67      0.59      0.60       188
weighted avg       0.72      0.76      0.72       188


AGE 25-44
Accuracy for age_25-44=1: 0.6805555555555556
Confusion Matrix for age_25-44=1:
[[ 88 190]
 [ 86 500]]
Classification Report for age_25-44=1:
              precision    recall  f1-score   support

           0       0.51      0.32      0.39       278
           1       0.72      0.85      0.78       586

    accuracy                           0.68       864
   macro avg       0.62      0.58      0.59       864
weighted avg       0.65      0.68      0.66       864


AGE 45-64
Accuracy for age_45-64=1: 0.679

In [ ]:
# insurance types
print("\nMEDICAID")
evaluate_subgroup('INS_Medicaid', 1)
print("\nMEDICARE")
evaluate_subgroup('INS_Medicare', 1)
print("\nOTHER INSURANCE")
evaluate_subgroup('INS_Other', 1)


MEDICAID
Accuracy for INS_Medicaid=1: 0.6798029556650246
Confusion Matrix for INS_Medicaid=1:
[[ 70 145]
 [ 50 344]]
Classification Report for INS_Medicaid=1:
              precision    recall  f1-score   support

           0       0.58      0.33      0.42       215
           1       0.70      0.87      0.78       394

    accuracy                           0.68       609
   macro avg       0.64      0.60      0.60       609
weighted avg       0.66      0.68      0.65       609


MEDICARE
Accuracy for INS_Medicare=1: 0.6428017575600931
Confusion Matrix for INS_Medicare=1:
[[ 692  802]
 [ 580 1795]]
Classification Report for INS_Medicare=1:
              precision    recall  f1-score   support

           0       0.54      0.46      0.50      1494
           1       0.69      0.76      0.72      2375

    accuracy                           0.64      3869
   macro avg       0.62      0.61      0.61      3869
weighted avg       0.63      0.64      0.64      3869


OTHER INSURANCE
Accur

# Model with exclusion of protected attributes

In [ ]:
protected_attributes = ['gender', 'RACE_Asian', 'RACE_Black/African American',
                 'RACE_Hispanic/Latino', 'RACE_Other', 'RACE_White', 'age_18-24',
                 'age_25-44', 'age_45-64', 'age_65-88', 'age_89+', 'INS_Medicaid', 'INS_Medicare', 'INS_Other']

featuresNew = [feat for feat in features if feat not in protected_attributes]

X_without_protected = df[featuresNew]
y = df['los_category']

X_train_np, X_test_np, y_train, y_test = train_test_split(X_without_protected, y_encoded, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_train_resampled_np, y_train_resampled = smote.fit_resample(X_train_np, y_train)

In [ ]:
best_rf_model.fit(X_train_resampled_np, y_train_resampled)
y_pred_rf_np = best_rf_model.predict(X_test_np)

accuracy_np = accuracy_score(y_test, y_pred_rf_np)
print("Overall Accuracy of XGBoost Classifier without protected attributes:", accuracy_np)

conf_matrix_np = confusion_matrix(y_test, y_pred_rf_np)
print("Overall Confusion Matrix without protected attributes:")
print(conf_matrix_np)

label_encoder = LabelEncoder()
label_encoder.fit(y_encoded)

y_test_labels = label_encoder.inverse_transform(y_test)
y_pred_labels = label_encoder.inverse_transform(y_pred_rf_np)

class_report_labels = classification_report(y_test_labels, y_pred_labels)
print("Overall Classification Report without protected attributes:")
print(class_report_labels)

Overall Accuracy of XGBoost Classifier without protected attributes: 0.6595436436073886
Overall Confusion Matrix without protected attributes:
[[1420 1624]
 [1196 4043]]
Overall Classification Report without protected attributes:
              precision    recall  f1-score   support

           0       0.54      0.47      0.50      3044
           1       0.71      0.77      0.74      5239

    accuracy                           0.66      8283
   macro avg       0.63      0.62      0.62      8283
weighted avg       0.65      0.66      0.65      8283



In [ ]:
# calculating metrics per protected attribute
def calculate_protected_attribute_metrics(results, protected_attribute):
    if protected_attribute == 'gender':
        unique_values = results[protected_attribute].unique()
        for value in unique_values:
            subset = results[results[protected_attribute] == value]
            y_test_subset = subset['y_test']
            y_pred_subset = subset['y_pred']

            accuracy = accuracy_score(y_test_subset, y_pred_subset)
            conf_matrix = confusion_matrix(y_test_subset, y_pred_subset)

            y_test_labels = label_encoder.inverse_transform(y_test_subset)
            y_pred_labels = label_encoder.inverse_transform(y_pred_subset)
            class_report = classification_report(y_test_labels, y_pred_labels)

            gender_label = 'Female' if value == 1 else 'Male'
            print(f"Metrics for {protected_attribute} = {gender_label}:")
            print(f"Overall Accuracy: {accuracy}")
            print("Confusion Matrix:")
            print(conf_matrix)
            print("Classification Report:")
            print(class_report)
            print("\n")
    else:
        unique_values = results[protected_attribute].unique()
        for value in unique_values:
          if value == 1:
            subset = results[results[protected_attribute] == value]
            y_test_subset = subset['y_test']
            y_pred_subset = subset['y_pred']

            accuracy = accuracy_score(y_test_subset, y_pred_subset)
            conf_matrix = confusion_matrix(y_test_subset, y_pred_subset)

            y_test_labels = label_encoder.inverse_transform(y_test_subset)
            y_pred_labels = label_encoder.inverse_transform(y_pred_subset)
            class_report = classification_report(y_test_labels, y_pred_labels)

            print(f"Metrics for {protected_attribute} = {value}:")
            print(f"Overall Accuracy: {accuracy}")
            print("Confusion Matrix:")
            print(conf_matrix)
            print("Classification Report:")
            print(class_report)
            print("\n")

X_test_with_protected = df.loc[X_test_np.index][protected_attributes].reset_index(drop=True)
results = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred_rf_np})
results = results.join(X_test_with_protected)

for protected_attribute in protected_attributes:
    calculate_protected_attribute_metrics(results, protected_attribute)


Metrics for gender = Female:
Overall Accuracy: 0.6627140974967062
Confusion Matrix:
[[ 798  889]
 [ 647 2220]]
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.47      0.51      1687
           1       0.71      0.77      0.74      2867

    accuracy                           0.66      4554
   macro avg       0.63      0.62      0.63      4554
weighted avg       0.65      0.66      0.66      4554



Metrics for gender = Male:
Overall Accuracy: 0.6556717618664522
Confusion Matrix:
[[ 622  735]
 [ 549 1823]]
Classification Report:
              precision    recall  f1-score   support

           0       0.53      0.46      0.49      1357
           1       0.71      0.77      0.74      2372

    accuracy                           0.66      3729
   macro avg       0.62      0.61      0.62      3729
weighted avg       0.65      0.66      0.65      3729



Metrics for RACE_Asian = 1:
Overall Accuracy: 0.6877192982456141
Confusion M

# Bias Mitigation


In [ ]:
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing

# Pre-processing Technique: weighting

Code inspo from: https://github.com/Trusted-AI/AIF360/blob/main/examples/demo_reweighing_preproc.ipynb


In [ ]:
train_features = ['previous_stays', 'prev_los_avg', 'Anion Gap',
                  'Bicarbonate', 'Chloride', 'Creatinine', 'Glucose', 'Hematocrit',
                  'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Platelet Count',
                  'Potassium', 'RDW', 'Red Blood Cells', 'Sodium', 'Urea Nitrogen',
                  'White Blood Cells', 'prescrip_count', 'diagnoses_num',
                  'ADM_Emergency', 'ADM_Other', 'ADM_Referral', 'ADM_Transfer',
                  'gender', 'RACE_Asian', 'RACE_Black/African American',
                  'RACE_Hispanic/Latino', 'RACE_Other', 'RACE_White', 'age_18-24',
                  'age_25-44', 'age_45-64', 'age_65-88', 'age_89+', 'INS_Medicaid', 'INS_Medicare', 'INS_Other']

# protected attributes
protected_attributes = ['gender', 'RACE_Asian', 'RACE_Black/African American',
                        'RACE_Hispanic/Latino', 'RACE_Other', 'RACE_White', 'age_18-24',
                        'age_25-44', 'age_45-64', 'age_65-88', 'age_89+', 'INS_Medicaid', 'INS_Medicare', 'INS_Other']

unprivileged_groups = [{'RACE_Other': 1}, {'RACE_Black/African American': 1},
                       {'age_18-24': 1}, {'age_25-44': 1}, {'age_89+': 1},
                       {'INS_Medicare': 1}, {'INS_Other': 1} ]


privileged_groups = [{'RACE_White': 1}, {'RACE_Hispanic/Latino': 1}, {'RACE_Asian': 1},
                     {'age_45-64': 1}, {'age_65-88': 1},
                     {'INS_Medicaid': 1}]


df['los_category'] = df_train['los_category'].map({'long': 0, 'short': 1})

In [ ]:
X = df[train_features]
y = df['los_category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Converting the training set into a BinaryLabelDataset
df_train = X_train.copy()
df_train['los_category'] = y_train


binary_label_dataset_train = BinaryLabelDataset(
    df=df_train,
    label_names=['los_category'],
    protected_attribute_names=protected_attributes
)

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

# reweighing on training set
train_rw = RW.fit_transform(binary_label_dataset_train)

df_train_rw = train_rw.convert_to_dataframe()[0]
X_train_rw = df_train_rw[train_features]
y_train_rw = df_train_rw['los_category']

# SMOTE for 'los' imbalance in reweighted training set
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_rw, y_train_rw)

In [ ]:
rf_model = RandomForestClassifier()

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [3, 6, 9],
    'max_features': [0.6, 0.7, 0.8]
}

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train_resampled, y_train_resampled)
print(f"Best parameters found: {grid_search.best_params_}")

best_rf_model = grid_search.best_estimator_
best_rf_model.fit(X_train_resampled, y_train_resampled)
y_pred_rf = best_rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rf)
print("Overall Accuracy of Random Forest Classifier:", accuracy)

conf_matrix = confusion_matrix(y_test, y_pred_rf)
print("Overall Confusion Matrix:")
print(conf_matrix)

label_encoder = LabelEncoder()
label_encoder.fit(y_encoded)

y_test_labels = label_encoder.inverse_transform(y_test)
y_pred_labels = label_encoder.inverse_transform(y_pred_rf)

class_report_labels = classification_report(y_test_labels, y_pred_labels)
print("Overall Classification Report:")
print(class_report_labels)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Best parameters found: {'max_depth': 9, 'max_features': 0.6, 'n_estimators': 100}
Overall Accuracy of Random Forest Classifier: 0.6672703126886393
Overall Confusion Matrix:
[[1270 1774]
 [ 982 4257]]


NameError: name 'y_encoded' is not defined

In [ ]:
class_report_labels = classification_report(y_test, y_pred_rf)
print("Overall Classification Report:")
print(class_report_labels)

Overall Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.42      0.48      3044
           1       0.71      0.81      0.76      5239

    accuracy                           0.67      8283
   macro avg       0.63      0.61      0.62      8283
weighted avg       0.65      0.67      0.65      8283



In [ ]:
def evaluate_subgroup(subgroup_name, subgroup_value):
    subgroup_indices = X_test[subgroup_name] == subgroup_value
    subgroup_X_test = X_test[subgroup_indices]
    subgroup_y_test = y_test[subgroup_indices]
    subgroup_y_pred = y_pred_rf[subgroup_indices]

    accuracy = accuracy_score(subgroup_y_test, subgroup_y_pred)
    print(f"Accuracy for {subgroup_name}={subgroup_value}: {accuracy}")

    conf_matrix = confusion_matrix(subgroup_y_test, subgroup_y_pred)
    print(f"Confusion Matrix for {subgroup_name}={subgroup_value}:")
    print(conf_matrix)

    class_report = classification_report(subgroup_y_test, subgroup_y_pred)
    print(f"Classification Report for {subgroup_name}={subgroup_value}:")
    print(class_report)

print("\nASIAN")
evaluate_subgroup('RACE_Asian', 1)
print("\nBLACK/AFRICAN AMERICAN")
evaluate_subgroup('RACE_Black/African American', 1)
print("\nHISPANIC/LATINO")
evaluate_subgroup('RACE_Hispanic/Latino', 1)
print("\nOTHER RACE")
evaluate_subgroup('RACE_Other', 1)
print("\nWHITE")
evaluate_subgroup('RACE_White', 1)


ASIAN
Accuracy for RACE_Asian=1: 0.6982456140350877
Confusion Matrix for RACE_Asian=1:
[[ 37  70]
 [ 16 162]]
Classification Report for RACE_Asian=1:
              precision    recall  f1-score   support

           0       0.70      0.35      0.46       107
           1       0.70      0.91      0.79       178

    accuracy                           0.70       285
   macro avg       0.70      0.63      0.63       285
weighted avg       0.70      0.70      0.67       285


BLACK/AFRICAN AMERICAN
Accuracy for RACE_Black/African American=1: 0.6862944162436548
Confusion Matrix for RACE_Black/African American=1:
[[135 210]
 [ 99 541]]
Classification Report for RACE_Black/African American=1:
              precision    recall  f1-score   support

           0       0.58      0.39      0.47       345
           1       0.72      0.85      0.78       640

    accuracy                           0.69       985
   macro avg       0.65      0.62      0.62       985
weighted avg       0.67      0.

In [ ]:
#  age groups
print("\nAGE 18-24")
evaluate_subgroup('age_18-24', 1)
print("\nAGE 25-44")
evaluate_subgroup('age_25-44', 1)
print("\nAGE 45-64")
evaluate_subgroup('age_45-64', 1)
print("\nAGE 65-88")
evaluate_subgroup('age_65-88', 1)
print("\nAGE 89+")
evaluate_subgroup('age_89+', 1)


AGE 18-24
Accuracy for age_18-24=1: 0.7659574468085106
Confusion Matrix for age_18-24=1:
[[ 10  39]
 [  5 134]]
Classification Report for age_18-24=1:
              precision    recall  f1-score   support

           0       0.67      0.20      0.31        49
           1       0.77      0.96      0.86       139

    accuracy                           0.77       188
   macro avg       0.72      0.58      0.59       188
weighted avg       0.75      0.77      0.72       188


AGE 25-44
Accuracy for age_25-44=1: 0.6967592592592593
Confusion Matrix for age_25-44=1:
[[ 85 193]
 [ 69 517]]
Classification Report for age_25-44=1:
              precision    recall  f1-score   support

           0       0.55      0.31      0.39       278
           1       0.73      0.88      0.80       586

    accuracy                           0.70       864
   macro avg       0.64      0.59      0.60       864
weighted avg       0.67      0.70      0.67       864


AGE 45-64
Accuracy for age_45-64=1: 0.677

In [ ]:
# insurance types
print("\nMEDICAID")
evaluate_subgroup('INS_Medicaid', 1)
print("\nMEDICARE")
evaluate_subgroup('INS_Medicare', 1)
print("\nOTHER INSURANCE")
evaluate_subgroup('INS_Other', 1)


MEDICAID
Accuracy for INS_Medicaid=1: 0.7110016420361248
Confusion Matrix for INS_Medicaid=1:
[[ 77 138]
 [ 38 356]]
Classification Report for INS_Medicaid=1:
              precision    recall  f1-score   support

           0       0.67      0.36      0.47       215
           1       0.72      0.90      0.80       394

    accuracy                           0.71       609
   macro avg       0.70      0.63      0.63       609
weighted avg       0.70      0.71      0.68       609


MEDICARE
Accuracy for INS_Medicare=1: 0.6461617989144481
Confusion Matrix for INS_Medicare=1:
[[ 677  817]
 [ 552 1823]]
Classification Report for INS_Medicare=1:
              precision    recall  f1-score   support

           0       0.55      0.45      0.50      1494
           1       0.69      0.77      0.73      2375

    accuracy                           0.65      3869
   macro avg       0.62      0.61      0.61      3869
weighted avg       0.64      0.65      0.64      3869


OTHER INSURANCE
Accur

## Exp gradient: in-processing


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

In [ ]:
train_features = ['previous_stays', 'prev_los_avg', 'Anion Gap',
                  'Bicarbonate', 'Chloride', 'Creatinine', 'Glucose', 'Hematocrit',
                  'Hemoglobin', 'MCH', 'MCHC', 'MCV', 'Magnesium', 'Platelet Count',
                  'Potassium', 'RDW', 'Red Blood Cells', 'Sodium', 'Urea Nitrogen',
                  'White Blood Cells', 'prescrip_count', 'diagnoses_num',
                  'ADM_Emergency', 'ADM_Other', 'ADM_Referral', 'ADM_Transfer',
                  'gender', 'RACE_Asian', 'RACE_Black/African American',
                  'RACE_Hispanic/Latino', 'RACE_Other', 'RACE_White', 'age_18-24',
                  'age_25-44', 'age_45-64', 'age_65-88', 'age_89+', 'INS_Medicaid', 'INS_Medicare', 'INS_Other']

protected_attributes = ['RACE_Asian', 'RACE_Black/African American',
                        'RACE_Hispanic/Latino', 'RACE_Other', 'RACE_White', 'age_18-24',
                        'age_25-44', 'age_45-64', 'age_65-88', 'age_89+', 'INS_Medicaid', 'INS_Medicare', 'INS_Other']

In [ ]:
X = df[train_features]
y = df['los_category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
X_train_resampled = pd.concat([X_train_resampled], axis=1)

rf_model = RandomForestClassifier(
    max_depth=9,
    max_features=0.6,
    n_estimators=100
)

# Fairness constraint: Demographic Parity
constraint = DemographicParity()

mitigator = ExponentiatedGradient(estimator=rf_model, constraints=constraint)
mitigator.fit(X_train_resampled, y_train_resampled, sensitive_features=X_train_resampled[protected_attributes])

y_pred = mitigator.predict(X_test)
print(y_pred)

/usr/local/lib/python3.10/dist-packages/fairlearn/reductions/_moments/utility_parity.py:212: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.pos_basis[i] = 0 + zero_vec
/usr/local/lib/python3.10/dist-packages/fairlearn/reductions/_moments/utility_parity.py:213: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.neg_basis[i] = 0 + zero_vec
/usr/local/lib/python3.10/dist-packages/fairlearn/reductions/_moments/utility_parity.py:212: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame

[0 0 0 ... 1 0 0]


In [ ]:
class_report_labels = classification_report(y_test, y_pred)

print("Overall Classification Report:")
print(class_report_labels)

Overall Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.57      0.49      3044
           1       0.69      0.56      0.62      5239

    accuracy                           0.56      8283
   macro avg       0.56      0.56      0.55      8283
weighted avg       0.59      0.56      0.57      8283



In [ ]:
def evaluate_subgroup(subgroup_name, subgroup_value):
    subgroup_indices = X_test[subgroup_name] == subgroup_value
    subgroup_X_test = X_test[subgroup_indices]
    subgroup_y_test = y_test[subgroup_indices]
    subgroup_y_pred = y_pred[subgroup_indices]

    accuracy = accuracy_score(subgroup_y_test, subgroup_y_pred)
    print(f"Accuracy for {subgroup_name}={subgroup_value}: {accuracy}")

    conf_matrix = confusion_matrix(subgroup_y_test, subgroup_y_pred)
    print(f"Confusion Matrix for {subgroup_name}={subgroup_value}:")
    print(conf_matrix)

    class_report = classification_report(subgroup_y_test, subgroup_y_pred)
    print(f"Classification Report for {subgroup_name}={subgroup_value}:")
    print(class_report)

print("\nASIAN")
evaluate_subgroup('RACE_Asian', 1)
print("\nBLACK/AFRICAN AMERICAN")
evaluate_subgroup('RACE_Black/African American', 1)
print("\nHISPANIC/LATINO")
evaluate_subgroup('RACE_Hispanic/Latino', 1)
print("\nOTHER RACE")
evaluate_subgroup('RACE_Other', 1)
print("\nWHITE")
evaluate_subgroup('RACE_White', 1)


ASIAN
Accuracy for RACE_Asian=1: 0.5578947368421052
Confusion Matrix for RACE_Asian=1:
[[ 58  49]
 [ 77 101]]
Classification Report for RACE_Asian=1:
              precision    recall  f1-score   support

           0       0.43      0.54      0.48       107
           1       0.67      0.57      0.62       178

    accuracy                           0.56       285
   macro avg       0.55      0.55      0.55       285
weighted avg       0.58      0.56      0.56       285


BLACK/AFRICAN AMERICAN
Accuracy for RACE_Black/African American=1: 0.5411167512690356
Confusion Matrix for RACE_Black/African American=1:
[[189 156]
 [296 344]]
Classification Report for RACE_Black/African American=1:
              precision    recall  f1-score   support

           0       0.39      0.55      0.46       345
           1       0.69      0.54      0.60       640

    accuracy                           0.54       985
   macro avg       0.54      0.54      0.53       985
weighted avg       0.58      0.

In [ ]:
#  age groups
print("\nAGE 18-24")
evaluate_subgroup('age_18-24', 1)
print("\nAGE 25-44")
evaluate_subgroup('age_25-44', 1)
print("\nAGE 45-64")
evaluate_subgroup('age_45-64', 1)
print("\nAGE 65-88")
evaluate_subgroup('age_65-88', 1)
print("\nAGE 89+")
evaluate_subgroup('age_89+', 1)


AGE 18-24
Accuracy for age_18-24=1: 0.5585106382978723
Confusion Matrix for age_18-24=1:
[[28 21]
 [62 77]]
Classification Report for age_18-24=1:
              precision    recall  f1-score   support

           0       0.31      0.57      0.40        49
           1       0.79      0.55      0.65       139

    accuracy                           0.56       188
   macro avg       0.55      0.56      0.53       188
weighted avg       0.66      0.56      0.59       188


AGE 25-44
Accuracy for age_25-44=1: 0.5636574074074074
Confusion Matrix for age_25-44=1:
[[159 119]
 [258 328]]
Classification Report for age_25-44=1:
              precision    recall  f1-score   support

           0       0.38      0.57      0.46       278
           1       0.73      0.56      0.64       586

    accuracy                           0.56       864
   macro avg       0.56      0.57      0.55       864
weighted avg       0.62      0.56      0.58       864


AGE 45-64
Accuracy for age_45-64=1: 0.5800970

In [ ]:
# insurance types
print("\nMEDICAID")
evaluate_subgroup('INS_Medicaid', 1)
print("\nMEDICARE")
evaluate_subgroup('INS_Medicare', 1)
print("\nOTHER INSURANCE")
evaluate_subgroup('INS_Other', 1)


MEDICAID
Accuracy for INS_Medicaid=1: 0.5779967159277504
Confusion Matrix for INS_Medicaid=1:
[[123  92]
 [165 229]]
Classification Report for INS_Medicaid=1:
              precision    recall  f1-score   support

           0       0.43      0.57      0.49       215
           1       0.71      0.58      0.64       394

    accuracy                           0.58       609
   macro avg       0.57      0.58      0.56       609
weighted avg       0.61      0.58      0.59       609


MEDICARE
Accuracy for INS_Medicare=1: 0.5482036701990178
Confusion Matrix for INS_Medicare=1:
[[ 850  644]
 [1104 1271]]
Classification Report for INS_Medicare=1:
              precision    recall  f1-score   support

           0       0.44      0.57      0.49      1494
           1       0.66      0.54      0.59      2375

    accuracy                           0.55      3869
   macro avg       0.55      0.55      0.54      3869
weighted avg       0.58      0.55      0.55      3869


OTHER INSURANCE
Accur